# Step 0 Set up `kgtk`
Use GitHub branch `dev`:

In [ ]:
%%bash
git clone -b dev https://github.com/usc-isi-i2/kgtk.git
cd kgtk
python setup.py install

# Step 1 Send a SPARQL query use `kgtk`

Example 1: Find spouse(s) of politicians:

### Define alias and variables

In [ ]:
%%bash
# Database
CLAIMS=../data/claims.tsv.gz
WIKI_INFO=../data/wikidata_infobox.tsv
# Results
RESULTS=../data/spouse_of_politician_kgtk.tsv
NEW_RESULTS=../data/new_spouse_of_politician.tsv
# Datatypes
NUMBERS=../data/numbers.tsv
STRINGS=../data/strings.tsv
STRUCTURED_LITERALS=../data/structued_literals.tsv
QNODES=../data/qnodes.tsv

### Main `kgtk` query:

In [1]:
# SPARQL query: 
# SELECT ?politicianLabel ?spouseLabel 
# WHERE
# { 
#   ?politician wdt:P31  wd:Q5 ; 
#               wdt:P106 wd:Q82955 ; 
#               wdt:P26  ?spouse . 
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
# } 
# where `P106` means "occupation", `Q82955` means "politician", `P26` means "spouse";

In [ ]:
%%bash
kgtk query -i $CLAIMS \
           --match 'c: (:Q82955)<-[:P106]-(politician)-[:P26]->(s)' \
           --return 'politician, "P26" as label, s' \
           -o $RESULTS

### Count known results in Wikidata database:

Count politician-spouse pairs / **rows** (result should -1 which is the header):

In [6]:
%%bash
wc -l $RESULTS

36975 data/spouse_of_politician_kgtk.tsv


Count how many **unique politicians** have spouse in Wikidata:

In [7]:
%%bash
kgtk query -i $RESULTS \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
30400


### Find unknow results in Wikidata database:
- Find all politicians

In [23]:
%%bash
kgtk query -i $CLAIMS \
           --match 'c: (:Q82955)<-[:P106]-(politician)' \
           --return 'politician, "P106" as label, "Q82955" as node2' \
           -o ../data/politician.tsv

- Eliminate politicians who have spouse(s)

In [25]:
%%bash
kgtk ifnotexists -i ../data/politician.tsv \
                 --filter-on ../data/spouse_of_politician_kgtk.tsv \
                 --input-keys node1 \
                 --filter-keys node1 \
                 -o ../data/politician_wo_spouse.tsv

### Count unknown results in Wikidata database:

In [1]:
%%bash
kgtk query -i ../data/politician_wo_spouse.tsv \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
619168


# Step 2 Infer properties

Use query results from Wikidata database to infer properties in Wikidata infobox and return the most frequent property.

In [41]:
%%bash
kgtk query -i $RESULTS -i $WIKI_INFO \
           --match 's: (n)-[]->(v), w: (n)-[p]->(v)' \
           --return 'p.label, count(v) as N' \
           --order-by 'N desc' \
           --limit 1

label	N
property:spouse	5697


# Step 3 Run query in Wikidata infobox 

For those don't have spouse, query in Wikidata infobox:

In [28]:
%%bash
WIKI_INFO=../data/wikidata_infobox.tsv
QUERY_FILE=../data/politician_wo_spouse.tsv
NEW_RESULTS=../data/new_spouse_of_politician.tsv

kgtk query -i $QUERY_FILE -i $WIKI_INFO \
           --match 'p: (politician)-[]->(), w: (politician)-[property]->(spouse)' \
           --where 'property.label = "property:spouse"' \
           --return 'politician, property.label, spouse' \
           -o $NEW_RESULTS

- Count rows of new findings:

In [1]:
%%bash
wc -l ../data/new_spouse_of_politician.tsv

58739 ../data/new_spouse_of_politician.tsv


- Count unique politicians of new findings:

In [30]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n: (p)-[]->()' \
           --return 'count(distinct p) as N'

N
39443


# Step 4 Datatype distribution of new findings

### 1. Numbers:

- Rows:

In [42]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n: (p)-[]->(s)' \
           --where 'kgtk_number(s)' \
           -o ../data/numbers.tsv

wc -l ../data/numbers.tsv

7336 data/numbers.tsv


- Unique politicians:

In [43]:
%%bash
kgtk query -i ../data/numbers.tsv \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
4587


- Duplicates (politicians that have multiple numbers):

In [ ]:
%%bash
kgtk query -i ../data/numbers.tsv \
           --match '(p)-[]->(s)' \
           --return 'p, count(s) as N' \
           --order-by 'N desc'

### 2. Structured literals:

- Rows:

In [44]:
%%bash
NEW_RESULTS=../data/new_spouse_of_politician.tsv
WIKI_INFO=../data/wikidata_infobox.tsv

kgtk query -i $NEW_RESULTS -i $WIKI_INFO \
           --match 'n: (q)-[p]->(s), w: (s)-[sv]->(v)' \
           --where 'NOT kgtk_lqstring(s) AND NOT kgtk_number(s) AND sv.label = "dbpedia:structured_value"' \
           --return 'q, p.label, v' \
           -o ../data/structured_literals.tsv

wc -l ../data/structured_literals.tsv

1943 data/structured_literals.tsv


- Unique politicians:

In [46]:
%%bash
kgtk query -i ../data/structured_literals.tsv \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
1942


- Duplicates:

In [ ]:
%%bash
kgtk query -i ../data/structured_literals.tsv \
           --match '(p)-[]->(s)' \
           --return 'p, count(s) as N' \
           --order-by 'N desc'

### 3. Strings:

#### All:

- Rows:

In [3]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n: (p)-[]->(s)' \
           --where 'kgtk_lqstring(s)' \
           -o ../data/strings.tsv

wc -l ../data/strings.tsv

48171 ../data/strings.tsv


- Unique politicians:

In [4]:
%%bash
kgtk query -i ../data/strings.tsv \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
38317


- Duplicates:

In [ ]:
%%bash
kgtk query -i ../data/strings.tsv \
           --match '(p)-[]->(s)' \
           --return 'p, count(s) as N' \
           --order-by 'N desc'

#### Empty:

- Rows:

In [13]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n: (p)-[]->(s)' \
           --where 'kgtk_lqstring_text(s) = ""' \
           -o ../data/empty_strings.tsv

wc -l ../data/empty_strings.tsv

5868 ../data/empty_strings.tsv


- Unique politicians:

In [14]:
%%bash
kgtk query -i ../data/empty_strings.tsv \
           --match 'n: (p)-[]->()' \
           --return 'count(distinct p) as N'

N
5867


which means there is no duplicate for empty string politician results.

#### Further check empty strings:
Since some of the empty items not excatly mean only return empty for that politician, for example, `Q1133864` has three spouse values: `nodemxZbyK2VRrGoaxfdLmyLxw-7343552`, `'Ethel Arnold'@en`, and `''@en`, so it need further check.

First we filter out all non-empty strings:

In [15]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n: (p)-[]->(s)' \
           --where 'NOT kgtk_lqstring(s) OR kgtk_lqstring_text(s) != ""' \
           -o ../data/non_empty_strings.tsv

Then we filter those in empty but also have spouse in non-empty ones, then we can count how many "pure" empty:

In [16]:
%%bash
kgtk ifnotexists -i ../data/empty_strings.tsv \
                 --filter-on ../data/non_empty_strings.tsv \
                 --input-keys node1 \
                 --filter-keys node1 \
                 -o ../data/pure_empty.tsv 

wc -l ../data/pure_empty.tsv

17 ../data/pure_empty.tsv


In [17]:
%%bash
kgtk query -i ../data/pure_empty.tsv \
           --match 'p: (politician)-[]->()' \
           --return 'count(distinct politician) as N'

N
16


Check one of them by hand:

In [18]:
%%bash
head -n 2 ../data/pure_empty.tsv

node1	label	node2
Q1594657	property:spouse	''@en


In [19]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match '(q:Q1594657)-[p]->(v)' \
           --return 'q, p.label, v'

node1	label	node2
Q1594657	property:spouse	''@en


### 4. Qnodes

- Rows:

In [2]:
%%bash
kgtk query -i ../data/new_spouse_of_politician.tsv \
           --match 'n:()-[]->(q)' \
           --where 'NOT kgtk_lqstring(q) AND NOT kgtk_number(q)' \
           -o ../data/nodes.tsv

kgtk ifnotexists -i ../data/nodes.tsv \
                 --filter-on ../data/structured_literals.tsv \
                 --input-keys node1 \
                 --filter-keys node1 \
                 -o ../data/qnodes.tsv 

wc -l ../data/qnodes.tsv

3234 ../data/qnodes.tsv


- Unique politicians:

In [56]:
%%bash
kgtk query -i ../data/qnodes.tsv \
           --match '(p)-[]->()' \
           --return 'count(distinct p) as N'

N
1275


- Duplicates:

In [ ]:
%%bash
kgtk query -i ../data/qnodes.tsv \
           --match '(p)-[]->(s)' \
           --return 'p, count(s) as N' \
           --order-by 'N desc'